In [5]:
from model.utils import Logger as LOG
LOG.measure_perf = True # ! ENABLE PERFORMANCE MONITORING
from model.utils import monitor_perf

from model.sign_to_text import SignToText
from model.speech_to_text import SpeechToText
from deep_translator import GoogleTranslator

from media_processing.video import webm_bytes_to_tensor, tensor_to_webm_bytes, load_mp4video_from_file, VideoPipeline
from media_processing.video import write_webm_bytes_to_file, read_webm_bytes_from_file
from media_processing.audio import webm_to_waveform

import matplotlib.pyplot as plt
import numpy as np
from pandas import DataFrame as df

import uuid

red = '\033[91m'
ylw = '\033[93m'
grn = '\033[92m'
blk = '\033[0m'

color_map = [
    "#46bdc6",
    "#4285F4",
    "#ff6d01",
    # "#F4B400",
    "#0F9D58",
]

google_languages = {
            "US": "en",
            "UK": "en",
            "DE": "de",
            "DK": "da"}

/Users/nielsraunkjaer/Desktop/Bachelor/SltServer/server-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
train = [
    f'./experiment-data/experiment_vids/train{i}.mp4'
    for i in range(100)
]
load_video = lambda video_idx: load_mp4video_from_file(video_idx)

# Downsampling visualizations

In [7]:
def show_frame(video_array,idx=0):
    plt.imshow(video_array[idx].detach().numpy())
    
def show_seq(video_array, idx_offset):
    w = 35
    h = 35
    fig = plt.figure(figsize=(w, h))
    columns = 10
    rows = 1
    for i in range(1, columns*rows +1):
        fig.add_subplot(rows, columns, i)
        show_frame(video_array, idx=i + idx_offset)
    plt.show()

# Performance tests GPU [Sign2Text]

In [34]:
import torch

In [35]:
slt_model = SignToText(device = 'cuda' if torch.cuda.is_available() else 'cpu')

Loading state dicts individually
Training backbone from scratch
Training head network from scratch
Freezing up to block 4 in S3D backbone
Loading model from pretrained checkpoint!


In [ ]:
assert slt_model.model.device.type[:4] == 'cuda', "Not on GPU yet!"

In [10]:
pipe = VideoPipeline(W_in=260, H_in=210, WH_out=224, k_t=1.)

In [22]:
# ! when not performing benchmarks
# del slt_model

In [24]:
""" Takes in a list of videos and monitors performance on the full SLT pipeline, using GPU acceleration """
def test_suite_slt_gpu(videos, repetitions=1, pipe=None, slt_model=None):
    assert pipe is not None and slt_model is not None, "Missing VideoPipeline or Sign2Text model."
    test_suite_ID = uuid.uuid4()
    for rep in range(repetitions):
        print(f"{red}Test Suite [{test_suite_ID}] {ylw}@ {grn}ITER {rep+1}{blk}")
        for video_idx in videos:
            repetition_ID = uuid.uuid4()
            _video = load_video(video_idx)
            # ? get webm bytes, just like we receive via websocket
            data = tensor_to_webm_bytes(_video)

            # video = webm_bytes_to_tensor(data)
            video = monitor_perf(webm_bytes_to_tensor, 
                                 data, width = 224, height = 224, 
                                 test_suite_ID=test_suite_ID, 
                                 repetition_ID=repetition_ID, 
                                 _input=len(data))
            
            # video = video.to('cuda') # ! send to GPU
            video = monitor_perf(video.to, 'cuda', # ! send to GPU
                                test_suite_ID=test_suite_ID, 
                                 repetition_ID=repetition_ID,
                                 variation='Send Video To GPU',
                                 _input=video.shape[0])

            # video, num_frames = pipe(video, to_file=False, output_length=True)
            video, num_frames = monitor_perf(pipe.__call__, 
                                             video, to_file=False, output_length=True, 
                                             variation=f"VideoPipeline, k_t={pipe.k_t}", 
                                             _input=video.shape[0], 
                                             test_suite_ID=test_suite_ID, 
                                             repetition_ID=repetition_ID)

            # y = slt_model(video, num_frames)[0]
            y = monitor_perf(slt_model.inference, video, num_frames, 
                             variation="Sign2Text", 
                             _input=int(num_frames[0]), 
                             test_suite_ID=test_suite_ID, 
                             repetition_ID=repetition_ID)[0]

            lang_from, lang_to = "DE", "US"
            # y = GoogleTranslator(source=google_languages[lang_from], target=google_languages[lang_to]).translate(y)
            monitor_perf(GoogleTranslator(source=google_languages[lang_from], target=google_languages[lang_to]).translate, y, 
                         variation="Google Translate", 
                         test_suite_ID=test_suite_ID, 
                         repetition_ID=repetition_ID)
        LOG.perfmon_step()
    print(f"{red}Test Suite [{test_suite_ID}] {ylw}@ {grn}FINISHED{blk}")


In [ ]:
""" Reset metrics (stores backup) """
LOG.reset_perf_metrics()

""" Run performance tests for 3 values of k_t """
pipe.k_t = 1.
test_suite_slt_gpu(train, repetitions=1, slt_model=slt_model, pipe=pipe)
pipe.k_t = .5
test_suite_slt_gpu(train, repetitions=1, slt_model=slt_model, pipe=pipe)
pipe.k_t = .33333
test_suite_slt_gpu(train, repetitions=1, slt_model=slt_model, pipe=pipe)

""" Save .csv-file """
LOG.save_perf_metrics()

In [23]:
""" see dataframe as table """
LOG.perf_metrics

,Function Name,Variation,Input Dims,time,perf_counter,process_time,Test Suite ID,Repetition ID
0,webm_bytes_to_tensor,None,151052,0.059362,0.059362,0.018372,eadfa273-2f99-4b8a-b377-5cc29d6cde27,1d5c3ea3-72f1-4a69-bbea-dd226769fb95
1,__call__,"VideoPipeline, k_t=1.0",100,0.028024,0.028025,0.125492,eadfa273-2f99-4b8a-b377-5cc29d6cde27,1d5c3ea3-72f1-4a69-bbea-dd226769fb95
2,inference,Sign2Text,tensor(100.),3.151217,3.151194,7.446397,eadfa273-2f99-4b8a-b377-5cc29d6cde27,1d5c3ea3-72f1-4a69-bbea-dd226769fb95
3,translate,Google Translate,None,0.106313,0.106313,0.012794,eadfa273-2f99-4b8a-b377-5cc29d6cde27,1d5c3ea3-72f1-4a69-bbea-dd226769fb95


In [ ]:
""" manually save dataframe """
LOG.save_perf_metrics()

In [ ]:
""" reset dataframe - will save current contents to backups folder """ 
LOG.reset_perf_metrics()

### Delete model from memory:

In [25]:
# get the model out of memory :D
del slt_model